# SHIFT_ML_2026_COMPETITION

## Описание задачи

Ссылка на архив с данными

Данные представляют собой более миллиона записей с более чем 100 признаками и одной целевой переменной: «итоговый_статус_займа». Значения: 0 – выплачен, 1 – не выплачен. Задача – бинарная классификация.

В вашем распоряжении будет два датасета: один тренировочный (с целевой переменной), и один тестовый – без целевой переменной.

Также в архиве находится ноутбук baseline.ipynb с примером подготовки файла с ответом.

Постановка задачи
Вам предстоит помочь банку принять решение о выдаче займа клиентам. Помните, что одного лишь принятого решения мало – нужно уметь интерпретировать полученные результаты.

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from catboost import CatBoostClassifier

## Загрузка данных

In [2]:
test_path = './shift_ml_2026_test.csv'
train_path = './shift_ml_2026_train.csv'
test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)
print('train')
print(train_df.shape)
display(train_df.head())
print('test')
print(test_df.shape)
display(test_df.head())

C:\Temp\ipykernel_23264\2058811446.py:4: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)


train
(1210779, 109)


,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,68355089,1235000.0,3 года,11.99,41014.0,В,В1,инженер,10+ лет,ИПОТЕКА,...,7.7,0.0,0.0,15700850.0,1973750.0,3965000.0,1233350.0,NaN,Нет,Наличные
1,68341763,1000000.0,5 лет,10.78,21633.0,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,...,50.0,0.0,0.0,10920900.0,934800.0,310000.0,743850.0,NaN,Нет,Наличные
2,68426831,597500.0,3 года,13.44,20259.0,В,В3,ветеринарный техник,4 года,АРЕНДА,...,100.0,0.0,0.0,845000.0,639900.0,470000.0,200000.0,NaN,Нет,Наличные
3,68476668,1000000.0,3 года,9.17,31879.0,Б,Б2,вице-президент операций по набору персонала,10+ лет,ИПОТЕКА,...,100.0,0.0,0.0,19442600.0,5838100.0,1575000.0,2322600.0,NaN,Нет,Наличные
4,67275481,1000000.0,3 года,8.49,31563.0,Б,Б1,дорожному водителю,10+ лет,ИПОТЕКА,...,0.0,0.0,0.0,9669500.0,1396850.0,725000.0,1807200.0,NaN,Нет,Наличные


test
(134531, 108)


,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,85540387,450000.0,3 года,9.49,14413.0,Б,Б2,обслуживание клиентов,10+ лет,ИПОТЕКА,...,75.0,0.0,0.0,4282850.0,1180600.0,725000.0,1022000.0,NaN,Нет,Наличные
1,28112500,400000.0,3 года,6.03,12174.5,А,А1,помощник по правовым вопросам,5 лет,АРЕНДА,...,57.1,0.0,0.0,3340900.0,2381050.0,1260000.0,1548550.0,NaN,Нет,Наличные
2,65731570,1250000.0,3 года,12.05,41548.0,В,В1,специалист по анализу кредитоспособности,5 лет,ИПОТЕКА,...,16.7,0.0,0.0,11350200.0,1980650.0,1690000.0,1804450.0,NaN,Нет,Наличные
3,65874747,977500.0,5 лет,20.99,26439.5,Д,Д5,специальный специалист,3 года,ИПОТЕКА,...,66.7,1.0,0.0,9976550.0,1678600.0,470000.0,2139950.0,NaN,Нет,Наличные
4,57893355,520000.0,3 года,18.25,18865.0,Д,Д1,руководитель районного проекта,3 года,АРЕНДА,...,66.7,0.0,0.0,1953950.0,1272750.0,1125000.0,603950.0,NaN,Нет,Наличные


## Предобработка данных

Разбиение на `train` и `val`

In [3]:
target = 'итоговый_статус_займа'
y = train_df[target]
X = train_df.drop(columns=[target])

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


### Доля пропусков в данных

Рассчитываем долю пропущенных значений как для тренировочной, так и для тестовой выборок. 

Сортируем по убыванию доли пропущенных значений

In [4]:
missing_train = X_train.isna().mean()
missing_test  = test_df.isna().mean()

missing_table = pd.DataFrame({
    "train": missing_train,
    "test": missing_test
})

missing_table["diff"] = (missing_table.train - missing_table.test).abs()
with pd.option_context("display.max_rows", None, 
                       "display.max_columns", None,
                       "display.float_format", '{:.4f}'.format):
    display(missing_table.sort_values("train", ascending=False).head(30))

,train,test,diff
дата_следующей_выплаты,1.0000,1.0000,0.0000
кредитный_баланс_по_возоб_счетам,0.9862,0.9865,0.0003
совокупный_статус_подтверждения_доходов_заемщиков,0.9811,0.9811,0.0000
совокупный_пдн_заемщиков,0.9809,0.9809,0.0000
совокупный_доход_заемщиков,0.9809,0.9809,0.0000
кол-во_месяцев_с_последнего_займа,0.8301,0.8298,0.0003
кол-во_мес_с_последней_задолженности_по_карте,0.7627,0.7639,0.0013
кол-во_месяцев_с_последнего_нарушения,0.7369,0.7392,0.0023
кол-во_мес_с_последней_задолженности_по_возобновляемому_счету,0.6653,0.6671,0.0018
соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам,0.6545,0.6534,0.0011


Как мы можем видеть, доля пропущенных значений изменяется от 0 до 1, что закономерно. На данный момент сложно сказать, что делать с пропусками, анализ этого момента будет ниже. Есть столбцы, которые нужно явно удалить:
* `дата_следующей_выплаты` - нет значений
* `пени_за_дефолт` - нет значений в `test`

Категориальные столбцы

Выводим для каждого категориального столбца его
* Название
* Число уникальных значений
* Топ 5 самых частых значений

In [30]:
cat_cols = X_train.select_dtypes(exclude='number').columns
for cat_col in cat_cols:
    data = X_train[cat_col]
    print(cat_col, len(data.unique()))
    print(data.unique())
    print(data.value_counts(ascending=False).head(5))
print(target, len(y_train.unique()))
print(y_train.unique())

срок_займа 2
['5 лет' '3 года']
срок_займа
3 года    734600
5 лет     234023
Name: count, dtype: int64
рейтинг 7
['В' 'Б' 'Д' 'А' 'Г' 'Ж' 'Е']
рейтинг
Б    282750
В    275200
А    169139
Г    144497
Д     67411
Name: count, dtype: int64
допрейтинг 35
['В1' 'Б1' 'Д2' 'А5' 'А4' 'Б3' 'Г2' 'Б4' 'В5' 'Д5' 'А1' 'В2' 'Ж4' 'Б2'
 'Д3' 'А3' 'Г4' 'Г1' 'Б5' 'Е3' 'В3' 'Е1' 'А2' 'В4' 'Г5' 'Г3' 'Ж2' 'Е2'
 'Ж1' 'Д4' 'Д1' 'Е5' 'Е4' 'Ж3' 'Ж5']
допрейтинг
В1    61464
Б4    59989
Б5    59524
Б3    58833
В2    57202
Name: count, dtype: int64
профессия_заемщика 229882
['ответственный за бухгалтерский учет' 'федеральный агент'
 'медицинское обслуживание' ... 'менеджер воздушно-розыскный центр'
 'писатель тв' 'квалифицированный работник общего профиля']
профессия_заемщика
менеджер         18678
преподаватель    17892
владелец         11232
медсестра        11178
водитель          9162
Name: count, dtype: int64
стаж 12
['10+ лет' '3 года' '2 года' '< 1 года' '7 лет' '9 лет' '6 лет' nan
 '1 год' '5 лет' '4 года

Большинство столбцов действительно категориальные, которые имеет смысл использовать для обучения модели, но есть исключения:
* `профессия заемщика` - почти 230000 уникальных значений, есть более и менее популярные, поле сложно использовать, т.к. каждая отдельная профессия встречается не более чем в 20000 записей из 1200000.
* `дата_первого_займа` - это дата, нужно привести к типу даты

In [31]:
X_train['дата_первого_займа'] = pd.to_datetime(X_train['дата_первого_займа'], format='%M-%Y')
X_val['дата_первого_займа'] = pd.to_datetime(X_val['дата_первого_займа'], format='%M-%Y')

In [33]:
X_train['дата_первого_займа']

938963   1992-01-01 00:12:00
108797   2002-01-01 00:08:00
287505   2001-01-01 00:03:00
403826   2005-01-01 00:08:00
484031   2000-01-01 00:05:00
                 ...        
345339   2006-01-01 00:02:00
554872   1994-01-01 00:04:00
168256   2001-01-01 00:04:00
56035    1992-01-01 00:02:00
636513   1994-01-01 00:11:00
Name: дата_первого_займа, Length: 968623, dtype: datetime64[ns]

Удаление пропусков

In [5]:
drop_cols = []

for col in X_train.columns:
    if missing_train[col] == 1:
        drop_cols.append(col)
    elif col in missing_test and abs(missing_train[col]-missing_test[col]) > 0.3:
        drop_cols.append(col)
    elif X_train[col].nunique() <= 1:
        drop_cols.append(col)

X_train = X_train.drop(columns=drop_cols)
X_val   = X_val.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)

print("Удалено:", len(drop_cols))


Удалено: 7


In [6]:
high_missing = missing_train[missing_train > 0.2].index

def add_missing_flags(df, cols):
    df = df.copy()
    for c in cols:
        if c in df:
            df[c+"_miss"] = df[c].isna().astype(int)
    return df

X_train = add_missing_flags(X_train, high_missing)
X_val   = add_missing_flags(X_val, high_missing)
test_df = add_missing_flags(test_df, high_missing)


Заполнение некоторых полей по смыслу:
* '%месяц%' -> 999
* '%кол-во%' -> 0
* остальные -> медиана

In [7]:
month_cols = [c for c in X_train.columns if "месяц" in c]

for c in month_cols:
    X_train[c] = X_train[c].fillna(999)
    X_val[c]   = X_val[c].fillna(999)
    test_df[c] = test_df[c].fillna(999)

In [8]:
count_cols = [c for c in X_train.columns if "кол-во" in c]

for c in count_cols:
    X_train[c] = X_train[c].fillna(0)
    X_val[c]   = X_val[c].fillna(0)
    test_df[c] = test_df[c].fillna(0)


In [9]:
num_cols = X_train.select_dtypes(include='number').columns
cat_cols = X_train.select_dtypes(exclude='number').columns
med = X_train[num_cols].median()

X_train[num_cols] = X_train[num_cols].fillna(med)
X_val[num_cols]   = X_val[num_cols].fillna(med)
test_df[num_cols] = test_df[num_cols].fillna(med)


Функция для расчета Information Value

In [10]:
def calc_iv(df, feature, target, bins=10):
    d = df[[feature, target]].copy()
    
    try:
        if d[feature].nunique() > bins:
            d["bin"] = pd.qcut(d[feature], bins, duplicates='drop')
        else:
            d["bin"] = d[feature]
    except:
        return 0
    
    g = d.groupby("bin")[target].agg(["count","sum"])
    g.columns = ["total","bad"]
    g["good"] = g.total - g.bad
    
    g["bad_rate"]  = g.bad / g.bad.sum()
    g["good_rate"] = g.good / g.good.sum()
    
    g["woe"] = np.log((g.good_rate+1e-6)/(g.bad_rate+1e-6))
    g["iv"]  = (g.good_rate-g.bad_rate)*g.woe
    
    return g.iv.sum()


Вычисляем IV для числовых признаков

In [ ]:
tmp = X_train.copy()
tmp[target] = y_train

iv_scores = {}

for col in X_train.columns:
    if col in cat_cols:
        continue
    iv_scores[col] = calc_iv(tmp, col, target)

iv_series = pd.Series(iv_scores).sort_values(ascending=False);
display(iv_series.head(30))


C:\Temp\ipykernel_23264\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_23264\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_23264\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ip

процентная_ставка                                    0.444614
сумма_выплат_по_просрочкам                           0.211605
нижний_порог_рейтинга_заемщика                       0.122058
верхний_порог_рейтинга_заемщика                      0.122058
пдн                                                  0.072216
кол-во_открытых_счетов_за_2_года                     0.066176
суммарная_доступная_сумма_займа_по_картам            0.053564
средний_баланс_текущих_счетов                        0.046855
кол-во_счетов_за_посл_год                            0.046131
кредитный_лимит                                      0.042718
лимит_по_картам                                      0.036020
общая_сумма_на_счетах                                0.035654
сумма_займа                                          0.034848
кол-во_активных_возобновляемых_счетов                0.034826
кол-во_возобновляемых_счетов_с_балансом_более_0      0.033318
кол-во_месяцев_с_последнего_счета                    0.033276
id      

Отбор по IV

In [12]:
iv_selected = iv_series[iv_series > 0.02].index.tolist()
print("IV selected:", len(iv_selected))


IV selected: 31


Отбор по корреляции

In [13]:
corr = X_train[iv_selected].corr().abs()

upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

to_drop_corr = [
    column for column in upper.columns
    if any(upper[column] > 0.9)
]

final_features = [c for c in iv_selected if c not in to_drop_corr]

print("После корреляции:", len(final_features))


После корреляции: 26


In [19]:
X_train.isna().sum().sort_values(ascending=False).head(10)

совокупный_статус_подтверждения_доходов_заемщиков    950268
профессия_заемщика                                    61739
стаж                                                  56482
кол-во_счетов_без_нарушений                               0
кол-во_карт                                               0
кол-во_аннуитетных_счетов                                 0
кол-во_открытых_возобновляемых_счетов                     0
кол-во_возобновляемых_счетов                              0
кол-во_возобновляемых_счетов_с_балансом_более_0           0
id                                                        0
dtype: int64

In [24]:
tmp = X_train.copy()
tmp[target] = y_train
for cat_col in cat_cols:
    print(cat_col, round(calc_iv(tmp, cat_col, target), 4))
    

срок_займа 0.1728
рейтинг 0.4575
допрейтинг 0
профессия_заемщика 0
стаж 0
владение_жильем 0.0317
подтвержден_ли_доход 0.0557
цель_займа 0
регион 0
пос_стоп_фактор 0.0
юридический_статус 0.0
дата_первого_займа 0
первоначальный_статус_займа 0.0003
тип_займа 0.0014
совокупный_статус_подтверждения_доходов_заемщиков 0.0174
тип_предоставления_кредита 0.0
